In [2]:
import pandas as pd
import numpy as np
import random
import re
import os
import pickle
from tqdm import tqdm
from datetime import datetime
from datetime import timedelta

ecg_sheet = pd.read_csv('/path/to/ecg/data')
echo_sheet = pd.read_csv('/path/to/echo/data')

# narrow down size of target df by only including rhythm and Lead 1 (chosen arbitrarily)
target_df = ecg_sheet[ecg_sheet['Waveform_Type'] == 'Rhythm']
target_df = target_df[target_df['Lead'] == 'I']

In [1]:
def detect_date_format(date_str):
    # List of common date formats
    formats = [
        "%Y-%m-%d %H:%M:%S",      # Example: 2024-08-16 15:22:35
        "%Y-%m-%d",               # Example: 2024-08-16
        "%d/%m/%Y",               # Example: 16/08/2024
        "%m/%d/%Y",               # Example: 08/16/2024
        "%Y%m%d%H%M%S.%f",        # Example: 20171013100544.000
        "%Y%m%d %H%M%S",          # Example: 20171013 100544
    ]
    
    for fmt in formats:
        try:
            datetime.strptime(date_str, fmt)
            return fmt
        except ValueError:
            continue
    return None

def compare_dates(ecg_date, echo_date):

    # detect date format for ECG and convert to datetime
    ecg_date_format = detect_date_format(ecg_date)
    ecg_datetime = datetime.strptime(ecg_date, ecg_date_format)

    # detect date format for echo and convert to datetime
    echo_date_format = detect_date_format(echo_date)
    echo_datetime = datetime.strptime(echo_date, echo_date_format)

    date_diff = abs(ecg_datetime - echo_datetime)

    # if the max date is greater that the date 6 months after the min date, they are not within 6 months
    return date_diff > timedelta(days=180), ecg_datetime, echo_datetime, date_diff


In [3]:
# extract all unique mrns 
unique_MRNs = target_df['Muse_PatientID'].unique()
echo_dates = []
echo_files = []
MRNs = []
ecg_dates = []
ecg_files = []
date_diffs = []
diagnoses = []
for i in tqdm(range(len(unique_MRNs))):

    # loop through each unique MRN and store ecg files that are the closest to the echo dates
    Muse_ID = unique_MRNs[i]

    # find index in diagnostic sheet where MRN occurs
    indx = echo_sheet.index[echo_sheet['MRN'] == Muse_ID].tolist()

    # extract echo date from that index
    echo_date = echo_sheet['Baseline Echo Procedure  Date'][indx]

    # extract echo file ID from that index
    resid = echo_sheet.iloc[indx]['RESID'].astype(str)
    resid = resid.tolist()
    resid = resid[0]

    if len(resid) == 1:
        echo_filename = 'RESID00' + resid
    elif len(resid) == 2:
        echo_filename = 'RESID0' + resid
    else:
        echo_filename = 'RESID' + resid

    # define new df containing ECG files for specific MRN
    ecg_df = target_df[target_df['Muse_PatientID'] == Muse_ID]
    ecg_df.reset_index(inplace=True, drop=True)

    ecg_date_diffs = []
    focused_ecg_date = []
    focused_ecg_files = []

    # extract all dates for specific MRN using all rows with MRN
    for j in range(len(ecg_df)):
        # raw ECG date (this is a string)
        date = ecg_df['AcquisitionDate'][j]

        ecg_file = ecg_df['filename'][j]

        flag, ecg_datetime, echo_datetime, date_diff = compare_dates(date, echo_date.values[0])
        
        if flag: # if its greater than 6 months, skip, otherwise, append the date difference
            continue
        else:
            ecg_date_diffs.append(date_diff)
            focused_ecg_date.append(ecg_datetime)
            focused_ecg_files.append(ecg_file)

    # determine smallest difference between echo and ecg dates
    if len(ecg_date_diffs) == 0:
        continue
    min_date = min(ecg_date_diffs)

    # determine which index has the smallest date gap
    ind_min_date = ecg_date_diffs.index(min_date)

    # extract ECG file and date that is closest to Echo date
    final_ecg_date = focused_ecg_date[ind_min_date]
    final_file = focused_ecg_files[ind_min_date]

    echo_files.append(echo_filename)
    date_diffs.append(min_date)
    echo_dates.append(echo_datetime)
    MRNs.append(Muse_ID)
    ecg_dates.append(final_ecg_date)
    ecg_files.append(final_file)

100%|██████████| 455/455 [00:00<00:00, 718.54it/s]


In [5]:
ecg_data_info = pd.DataFrame(columns=['ECG_File', 'ECG_Date', 'MRN', 'Echo_File', 'Echo_Date', 'Min_Date_Diff'])
ecg_data_info['ECG_File'] = ecg_files
ecg_data_info['ECG_Date'] = ecg_dates
ecg_data_info['MRN'] = MRNs
ecg_data_info['Echo_File'] = echo_files
ecg_data_info['Echo_Date'] = echo_dates
ecg_data_info['Min_Date_Diff'] = date_diffs
ecg_data_info.to_csv('/path/to/save/csv')